![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)
![SSU AILAB Logo](http://ailab.ssu.ac.kr/rb/layouts/default/image/common/logo.png)

* ### Spark Context import 확인

In [3]:
type(sc)

pyspark.context.SparkContext

In [41]:
sc.__dict__

{'_accumulatorServer': <pyspark.accumulators.AccumulatorServer instance at 0x113925098>,
 '_batchSize': 0,
 '_callsite': CallSite(function='<module>', file='/Users/Namgee/anaconda/lib/python2.7/site-packages/IPython/utils/py3compat.py', linenum=288),
 '_conf': <pyspark.conf.SparkConf at 0x11391cf10>,
 '_javaAccumulator': JavaObject id=o12,
 '_jsc': JavaObject id=o9,
 '_pickled_broadcast_vars': set(),
 '_python_includes': [],
 '_temp_dir': u'/private/var/folders/vp/xk17mlgj6r90566s8v5k_rs80000gn/T/spark-3778e2f5-0a24-41b1-b373-1e9bc6864317/pyspark-f1af9738-dae5-4bd1-9292-d69e0e84d74f',
 '_unbatched_serializer': PickleSerializer(),
 'appName': u'PySparkShell',
 'environment': {},
 'master': u'local[*]',
 'profiler_collector': None,
 'pythonExec': 'python2.7',
 'pythonVer': '2.7',
 'serializer': AutoBatchedSerializer(PickleSerializer()),
 'sparkHome': None}

In [42]:
sc.appName = "TestApp"

In [43]:
sc.appName

'TestApp'

* ### 파일로 부터 Spark RDD 생성
(4개의 파티션 생성)

In [12]:
data = sc.textFile("/Users/MJ/Desktop/KBO_Teams.txt", 4)

* ### 파티션별 데이터 확인

In [13]:
data.glom().collect()

[[u'doosan$144$79$65$0$0.370$0.435',
  u'samsung$144$88$56$0$0.378$0.469',
  u'nc$144$84$57$3$0.367$0.455'],
 [u'nexen$144$78$65$1$0.372$0.486', u'sk$144$69$73$2$0.349$0.410'],
 [u'hanhwa$144$68$76$0$0.360$0.404',
  u'kia$144$67$77$0$0.326$0.392',
  u'lotte$144$66$77$1$0.356$0.446'],
 [u'lg$144$64$78$2$0.339$0.399', u'kt$144$52$91$1$0.345$0.402']]

* ### 파티션의 개수를 6개로 변경후 확인

In [14]:
data1 = data.repartition(6)

In [15]:
data1.glom().collect()

[[u'kia$144$67$77$0$0.326$0.392'],
 [u'doosan$144$79$65$0$0.370$0.435', u'lotte$144$66$77$1$0.356$0.446'],
 [u'samsung$144$88$56$0$0.378$0.469'],
 [u'nc$144$84$57$3$0.367$0.455', u'lg$144$64$78$2$0.339$0.399'],
 [u'nexen$144$78$65$1$0.372$0.486', u'kt$144$52$91$1$0.345$0.402'],
 [u'sk$144$69$73$2$0.349$0.410', u'hanhwa$144$68$76$0$0.360$0.404']]

In [28]:
print data1.getNumPartitions()

6


* ### xrange, range 차이점 (python 내용)
(xrange 는 lazy evaluation)

In [32]:
dataXrange = xrange(1,10)

In [33]:
type(dataXrange)

xrange

In [48]:
dataRange = range(11,21)

In [49]:
type(dataRange)

list

In [50]:
dataXrange

xrange(1, 10)

In [51]:
dataRange

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

* ### 객체로 부터 RDD 생성

In [52]:
xrangeRDD = sc.parallelize(dataXrange)

In [53]:
rangeRDD = sc.parallelize(dataRange)

* ### toDebugString 를 통한 RDD 의 lineage 확인

In [54]:
xrangeRDD.toDebugString()

'(8) PythonRDD[28] at RDD at PythonRDD.scala:43 []\n |  ParallelCollectionRDD[26] at parallelize at PythonRDD.scala:423 []'

In [55]:
rangeRDD.toDebugString()

'(8) ParallelCollectionRDD[27] at parallelize at PythonRDD.scala:423 []'

* ### xrange, range 타입에 따라 생성되는 RDD 의 타입 확인

In [56]:
print 'type of RangeRDD: {0}'.format(type(xrangeRDD))

type of RangeRDD: <class 'pyspark.rdd.PipelinedRDD'>


In [57]:
print 'type of XrangeRDD: {0}'.format(type(rangeRDD))

type of XrangeRDD: <class 'pyspark.rdd.RDD'>


In [58]:
print 'id : {0}'.format(rangeRDD.id())
print 'id : {0}'.format(xrangeRDD.id())

id : 27
id : 28


* ### function 을 정의하고 Spark 의 map 함수 적용

In [71]:
def sub(value):
    return (value - 1)

subRDD = xrangeRDD.map(sub)

print "원본 데이터 : ", xrangeRDD.glom().collect()
print "sub 데이터 : ",subRDD.glom().collect()
print "lineage : ", subRDD.toDebugString()

원본 데이터 :  [[1], [2], [3], [4], [5], [6], [7], [8, 9]]
sub 데이터 :  [[0], [1], [2], [3], [4], [5], [6], [7, 8]]
lineage :  (8) PythonRDD[49] at RDD at PythonRDD.scala:43 []
 |  ParallelCollectionRDD[26] at parallelize at PythonRDD.scala:423 []


* ### Spark의 filter 함수 적용 과 lambda 사용법

In [73]:
def five(value):
    if(value < 5):
        return True
    else:
        return False

filteredRDD = subRDD.filter(five)
filteredRDD2 = subRDD.filter(lambda x: x > 5)

print "sub 데이터 : ", subRDD.glom().collect()
print "Partition별 5보다 작은 데이터", filteredRDD.glom().collect()
print "Partition별 5보다 큰 데이터", filteredRDD2.glom().collect()
print "filteredRDD Partition 개수 : ", filteredRDD.getNumPartitions()
print "filteredRDD2 Partition 개수 : ", filteredRDD2.getNumPartitions()

sub 데이터 :  [[0], [1], [2], [3], [4], [5], [6], [7, 8]]
Partition별 5보다 작은 데이터 [[0], [1], [2], [3], [4], [], [], []]
Partition별 5보다 큰 데이터 [[], [], [], [], [], [], [6], [7, 8]]
filteredRDD Partition 개수 :  8
filteredRDD2 Partition 개수 :  8


* ### RDD의 first 함수는 첫번 째 partition 의 첫번 째 데이터를 Return, take(1)은 first() 와 동일, takeOrdered 는 오름차순 정렬 후 Return

In [75]:
text = sc.textFile("/Users/MJ/Desktop/KBO_Teams.txt", 4) \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.reduceByKey(lambda v1, v2: v1+v2)
print text.glom().collect()
print text.first() # First Partition의 첫번째 Element.
print text.take(6)
print text.takeOrdered(3) # Collect Action 후에 작은 Element 순으로.
print text.top(5) #Collect Action 후에 큰 Element 순으로.

[[(u'hanhwa$144$68$76$0$0.360$0.404', 1)], [(u'nc$144$84$57$3$0.367$0.455', 1), (u'sk$144$69$73$2$0.349$0.410', 1)], [(u'samsung$144$88$56$0$0.378$0.469', 1), (u'kt$144$52$91$1$0.345$0.402', 1), (u'doosan$144$79$65$0$0.370$0.435', 1)], [(u'nexen$144$78$65$1$0.372$0.486', 1), (u'kia$144$67$77$0$0.326$0.392', 1), (u'lg$144$64$78$2$0.339$0.399', 1), (u'lotte$144$66$77$1$0.356$0.446', 1)]]
(u'hanhwa$144$68$76$0$0.360$0.404', 1)
[(u'hanhwa$144$68$76$0$0.360$0.404', 1), (u'nc$144$84$57$3$0.367$0.455', 1), (u'sk$144$69$73$2$0.349$0.410', 1), (u'samsung$144$88$56$0$0.378$0.469', 1), (u'kt$144$52$91$1$0.345$0.402', 1), (u'doosan$144$79$65$0$0.370$0.435', 1)]
[(u'doosan$144$79$65$0$0.370$0.435', 1), (u'hanhwa$144$68$76$0$0.360$0.404', 1), (u'kia$144$67$77$0$0.326$0.392', 1)]
[(u'sk$144$69$73$2$0.349$0.410', 1), (u'samsung$144$88$56$0$0.378$0.469', 1), (u'nexen$144$78$65$1$0.372$0.486', 1), (u'nc$144$84$57$3$0.367$0.455', 1), (u'lotte$144$66$77$1$0.356$0.446', 1)]


* ### RDD의 reduce 함수를 적용(substract 를 사용시 주의)
reduce 는 function 이 associative 하고 commutative 해야한다.

In [96]:
from operator import add

print filteredRDD.glom().collect()
# - 는 associative 하지만, commutative하지 못하다.
print "reduce(sub) : ", filteredRDD.reduce(lambda a, b: a - b)

# partition에 따라 sub 함수의 값이 달라짐(commutative하지 못하기 때문)
rePart_FilteredRDD = filteredRDD.repartition(3)
print rePart_FilteredRDD.glom().collect()
print "reduce(sub) : ", rePart_FilteredRDD.reduce(lambda a, b: a - b)

# sum 은 partition 을 나눠도 associative 하고 commutative 하기 때문에 결과 같음
print "reduce(sum) : ", filteredRDD.reduce(lambda a, b: a + b)
print "reduce(sum) : ", filteredRDD.repartition(3).reduce(lambda a, b: a + b)

[[0], [1], [2], [3], [4], [], [], []]
reduce(sub) :  -10
[[3, 4], [0, 1], [2]]
reduce(sub) :  -2
reduce(sum) :  10
reduce(sum) :  10


* ### takeSample 함수를 사용하여 샘플 데이터 추출
(withReplacement 는 중복 허용 여부, num 는 샘플 개수, seed 를 주면 항상 같은 샘플 추출)

In [95]:
print filteredRDD.takeSample(withReplacement=True, num=6)
print filteredRDD.takeSample(withReplacement=False, num=6)
print filteredRDD.takeSample(withReplacement=False, num=6, seed=500)
print filteredRDD.takeSample(withReplacement=True, num=6, seed=500)

[0, 3, 2, 3, 0, 0]
[1, 3, 2, 4, 0]
[2, 0, 1, 4, 3]
[4, 2, 3, 4, 3, 1]


* ### Map 함수와 flatMap 차이점

In [139]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList,4)

sigularAndPluralWordsRDDMap = wordsRDD.map(lambda x: (x, x + 's'))
sigularAndPluralWordsRDD = wordsRDD.flatMap(lambda x: (x, x + 's'))

print sigularAndPluralWordsRDDMap.collect()
print sigularAndPluralWordsRDD.collect()
print sigularAndPluralWordsRDDMap.count()
print sigularAndPluralWordsRDD.count()

[('cat', 'cats'), ('elephant', 'elephants'), ('rat', 'rats'), ('rat', 'rats'), ('cat', 'cats')]
['cat', 'cats', 'elephant', 'elephants', 'rat', 'rats', 'rat', 'rats', 'cat', 'cats']
5
10


In [107]:
simpleRDD = sc.parallelize([2,3,4])
print simpleRDD.map(lambda x: range(1, x)).collect()
print simpleRDD.flatMap(lambda x: range(1, x)).collect()


[[1], [1, 2], [1, 2, 3]]
[1, 1, 2, 1, 2, 3]


* ### reduceByKey, groupByKey, countByValue 함수
(각각의 return type 숙지, groupByKey 사용법 등 주의)

![reduceByKey() figure](http://spark-mooc.github.io/web-assets/images/reduce_by.png)
![groupByKey() figure](http://spark-mooc.github.io/web-assets/images/group_by.png)

#### 1) pairRDD 타입에서의 groupByKey, reduceByKey, mapValues 사용법

In [142]:
pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 1)])
# mapValues only used to improve format for printing
print pairRDD.groupByKey().mapValues(lambda x: list(x)).collect()

# Different ways to sum by key
print pairRDD.groupByKey().map(lambda (k, v): (k, sum(v))).collect()
# Using mapValues, which is recommended when they key doesn't change
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect()
# reduceByKey is more efficient / scalable
print pairRDD.reduceByKey(add).collect()

[('a', [1, 2]), ('b', [1])]
[('a', 3), ('b', 1)]
[('a', 3), ('b', 1)]
[('a', 3), ('b', 1)]


#### 2) WordCount 비교

In [143]:
repetitiveRDD = sc.parallelize(["i","am", "boy", "i"])

text1 = repetitiveRDD \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.reduceByKey(lambda v1, v2: v1 + v2).collect() # collectAsMap() 사용 시 dictionary 형태로 return

print "reduceByKey : ", text1

text2 = repetitiveRDD \
.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
.groupByKey() \
.map(lambda (k, v): (k, sum(v))).collect() # groupByKey는 (key, resultIterable) 형태

print "groupByKey : ", text2

# text3 의 countByValue 는 dictionary 형태를 return
text3 = repetitiveRDD \
.flatMap(lambda line: line.split()).countByValue()

print "countByValue : ", text3

print "i 가 몇번? : ", [count for word, count in text2 if word == 'i'], "번"

reduceByKey :  [('i', 2), ('boy', 1), ('am', 1)]
groupByKey :  [('i', 2), ('boy', 1), ('am', 1)]
countByValue :  defaultdict(<type 'int'>, {'i': 2, 'boy': 1, 'am': 1})
i 가 몇번? :  [2] 번


* ### mapPartitions, mapPartitionsWithIndex 사용법
(mapPartitions : partition 끼리만 f 적용,
mapPartitionsWithIndex : partition 끼리만 f 적용하며 추가로 index 값을 생성해 준다.)

#### 1) join 예제

In [122]:
s = "-";
seq = ("a", "b", "c"); # This is sequence of strings.
print s.join( list(seq) )

a-b-c


In [147]:
print wordsRDD.glom().collect()
itemRDD = wordsRDD.mapPartitions(lambda iterator: [', '.join(iterator)])
print itemRDD.glom().collect() # 4번째 partition의 'rat'과 'cat' 만 join

itemByPartRDD = wordsRDD.mapPartitionsWithIndex(lambda index, iterator: \
                                                (index, list(iterator)))

itemByPartRDD1 = wordsRDD.mapPartitionsWithIndex(lambda index, iterator: \
                                                (index, [', '.join(iterator)]))
print itemByPartRDD.glom().collect()
print itemByPartRDD1.glom().collect()

[['cat'], ['elephant'], ['rat'], ['rat', 'cat']]
[['cat'], ['elephant'], ['rat'], ['rat, cat']]
[[0, ['cat']], [1, ['elephant']], [2, ['rat']], [3, ['rat', 'cat']]]
[[0, ['cat']], [1, ['elephant']], [2, ['rat']], [3, ['rat, cat']]]
